In [ ]:
import numpy as np
import os
import plotly.graph_objects as go

from chimera_fgo.util.kitti import process_kitti_gt, load_icp_results
from chimera_fgo.util.plot import plot_trajectories, plot_MC

%load_ext autoreload
%autoreload 2

In [ ]:
kitti_seq = '0018'
run_name = 'fgo_0m_100runs_blind_2023-01-19-2054'
results_path = os.path.join(os.getcwd(), '..', 'results', kitti_seq, run_name)
results_files = os.listdir(results_path)
fgo_blind_results = np.load(os.path.join(results_path, results_files[0]))

In [ ]:
#qs_plot = fgo_results['qs']
qs_plot = fgo_blind_results['qs']
T = fgo_blind_results['threshold']
GPS_RATE = 10
detect_idx = np.argmax(qs_plot > T)

# Plot test statistic and threshold
fig = go.Figure()
fig.add_trace(go.Scatter(x=np.arange(len(qs_plot)), y=qs_plot, name='Test statistic'))
fig.add_trace(go.Scatter(x=np.arange(len(qs_plot)), y=T*np.ones(len(qs_plot)), name='Threshold', line=dict(color='black', dash='dash')))
# Add vertical line at start of spoofing attack
fig.add_trace(go.Scatter(x=[100, 100], y=[min(qs_plot), max(qs_plot)], mode='lines', name='Start of attack', line=dict(color='red', dash='dash')))
# fig.add_shape(type="line", x0=ATTACK_START/GPS_RATE, y0=-20, x1=ATTACK_START/GPS_RATE, y1=200, name='Start of attack', 
#               line=dict(color="red", width=2, dash="dash"), showlegend=True)
fig.add_trace(go.Scatter(x=[detect_idx], y=[qs_plot[detect_idx]], name='Detection', mode='markers', marker=dict(size=10, color='red'), showlegend=True))
fig.update_layout(width=900, height=500, xaxis_title='Time [s]', yaxis_title='Test statistic')
fig.update_layout(legend=dict(x=0.05, y=0.98), font=dict(size=18))
fig.show()

In [ ]:
n_runs = len(results_files)

fig = go.Figure()

per_trial_FA = 0
per_run_FA = 0

for i in range(n_runs):
    results = np.load(os.path.join(results_path, results_files[i]))
    qs = results['qs']
    T = results['threshold']
    detect_idx = np.argmax(qs > T)
    showlegend = True if i == 0 else False
    fig.add_trace(go.Scatter(x=np.arange(len(qs)), y=qs, name='Test statistic', line=dict(color='blue'), showlegend=showlegend))
    #fig.add_trace(go.Scatter(x=[detect_idx], y=[qs[detect_idx]], name='Detection', mode='markers', marker=dict(size=10, color='red'), showlegend=showlegend))
    per_trial_FA += sum(qs > T)
    per_run_FA += 1 if sum(qs > T) > 0 else 0

fig.add_trace(go.Scatter(x=np.arange(len(qs)), y=T*np.ones(len(qs)), name='Threshold', line=dict(color='black', dash='dash')))
#fig.add_trace(go.Scatter(x=[100, 100], y=[50, 400], mode='lines', name='Start of attack', line=dict(color='red', dash='dash')))
fig.update_layout(width=900, height=500, xaxis_title='Time [s]', yaxis_title='Test statistic')
fig.update_layout(legend=dict(x=0.05, y=0.98), font=dict(size=18))
fig.update_yaxes(range=[50, 200])
fig.show()